In [72]:
import pandas as pd

In [73]:
path = './data/UtilizationRateInfo_4.xlsx'
test_div = '23D000'
test_project_code = '1PD05R550001'

df_all = pd.read_excel(path, engine='openpyxl')
print(df_all.shape)
df_all.head(2)

(11809, 26)


,emplid,div,deptid,sub_job_family,date,attendance,project_code,total_hour,pmcs_bu_start_dt,project_start_stage,...,C6_day,stage,termination_n,month_pf,execute_day,execute_hour,execute_month,utilization_rate_by_dep_func,utilization_rate_by_dep,utilization_rate_by_div_func
0,10005085,23M000,23M300,ME,2017-08-01,160,NaN,0.0,Microsoft.DPrep.ErrorValues.PythonNumpyDatetim...,NaN,...,NaN,NaN,0,NaN,2.979144e+06,0.0,100498.392934,0.356181,0.356181,0.357213
1,10005085,23M000,23M300,ME,2017-09-01,160,NaN,0.0,Microsoft.DPrep.ErrorValues.PythonNumpyDatetim...,NaN,...,NaN,NaN,0,NaN,2.979888e+06,0.0,100498.392934,0.393403,0.393403,0.391757


In [74]:
df_all['sub_job_family'] = df_all['sub_job_family'].apply(lambda x: 'RH' if x in ('AR', 'HW', 'OP', 'RF', 'RX') else x)
df_all['sub_job_family'].unique()

array(['ME', 'RH', 'RS'], dtype=object)

In [75]:
df_all2 = df_all.copy()

In [76]:
df = df_all[df_all['div'] == test_div]
print(df.shape)
df.head(2)

(1788, 26)


,emplid,div,deptid,sub_job_family,date,attendance,project_code,total_hour,pmcs_bu_start_dt,project_start_stage,...,C6_day,stage,termination_n,month_pf,execute_day,execute_hour,execute_month,utilization_rate_by_dep_func,utilization_rate_by_dep,utilization_rate_by_div_func
17,10006020,23D000,23D500,RH,2020-06-01,184,1PD042550001,0.5,2018-01-23 08:00:00,C1,...,92.0,C5,0,20.0,20656.0,89325.402194,1054.133333,1.129814,1.129814,0.488366
18,10006020,23D000,23D500,RH,2020-06-01,184,1PD05K550001,15.1,2019-09-12 08:00:00,C0,...,92.0,C2,0,5.0,7648.0,12888.947919,620.533333,1.129814,1.129814,0.488366


In [77]:
# from df 
dic_rename = {'utilization_rate_by_dep_func':'uti_rate', 'attendance':'attendance_emp',
     'total_hour':'total_hour_dept_func',
     'emplid':'emp_cnt', 'attendance':'attendance_emp'}

lst_groupby = ['div', 'deptid', 'sub_job_family', 'date']
dic_agg = {'utilization_rate_by_dep_func':'mean', 'attendance':'mean',
           'total_hour':'sum', 'termination_n':'sum', 'emplid': 'nunique'}
df_func_rate = df.groupby(lst_groupby).agg(dic_agg).rename(columns=dic_rename).reset_index()
print(df_func_rate.shape)
df_func_rate

(90, 9)


,div,deptid,sub_job_family,date,uti_rate,attendance_emp,total_hour_dept_func,termination_n,emp_cnt
0,23D000,23D000,RH,2020-05-01,0.000000,160,0.0,0,2
1,23D000,23D000,RH,2020-06-01,0.000000,184,0.0,0,2
2,23D000,23D000,RH,2020-07-01,0.000000,160,0.0,0,2
3,23D000,23D000,RH,2020-08-01,0.000000,200,0.0,0,2
4,23D000,23D000,RH,2020-09-01,0.000000,144,0.0,0,2
...,...,...,...,...,...,...,...,...,...
85,23D000,23D600,RH,2021-01-01,0.053125,160,8.5,0,1
86,23D000,23D600,RH,2021-02-01,0.208333,120,25.0,0,1
87,23D000,23D600,RH,2021-03-01,0.320652,184,59.0,0,1
88,23D000,23D600,RH,2021-04-01,0.545139,144,78.5,0,1


In [78]:
# from df_all
dic_agg = {'C0_day':'mean', 'C1_day':'mean','C2_day':'mean', 'C3_day':'mean',
           'C4_day':'mean', 'C5_day':'mean','C6_day':'mean', 'execute_hour':'mean',
           'execute_month':'mean', 'execute_day':'max','project_start_stage':'unique'}
date_min = min(df_all['date'])
print(date_min)
print('df_all project count:', len(df_all['project_code'].unique()))
print(df_all.shape)
df_proj = df_all.copy()[df_all.copy()['pmcs_bu_start_dt']!='Microsoft.DPrep.ErrorValues.PythonNumpyDatetimeParseFailure']
print(df_proj.shape)

df_proj['pmcs_bu_start_dt'] = df_proj['pmcs_bu_start_dt'].astype('datetime64[ns]')
print('df_proj project count:', len(df_proj['project_code'].unique()))
df_proj = df_proj[df_proj['pmcs_bu_start_dt'] > date_min]
print('df_proj project count:', len(df_proj['project_code'].unique()))
print(df_proj.shape)

df_proj = df_proj.groupby(['project_code']).agg(dic_agg).reset_index()
print(df_proj.shape)
df_proj.head(3)

2017-08-01 00:00:00
df_all project count: 119
(11809, 26)
(9365, 26)
df_proj project count: 118
df_proj project count: 70
(6139, 26)
(70, 12)


,project_code,C0_day,C1_day,C2_day,C3_day,C4_day,C5_day,C6_day,execute_hour,execute_month,execute_day,project_start_stage
0,1PD00T550001,0.0,0.0,0.0,0.0,0.0,729.0,91.0,768.8,657.333333,19720.0,[C5]
1,1PD02W27A001,8.0,9.0,32.0,42.0,28.0,2419.0,92.0,2247.5,570.133333,17104.0,[C0]
2,1PD032550001,18.0,53.0,89.0,55.0,63.0,818.0,91.0,573.4,262.933333,7888.0,[C0]


In [79]:
lst_column = ['project_code', 'execute_hour', 'execute_month', 'project_start_stage']
proj_prop = df_proj[df_proj['project_code']==test_project_code].loc[:, lst_column]
proj_prop['hr_ratio'] = 1 
proj_prop['mon_ratio'] = 1
proj_prop.head(2)

,project_code,execute_hour,execute_month,project_start_stage,hr_ratio,mon_ratio
30,1PD05R550001,59677.298815,513.333333,[C0],1,1


In [80]:
lst_groupby = ['project_code', 'sub_job_family', 'date']
dic_agg = {'total_hour':'sum'}
dic_rename = {'total_hour':'total_hour_pro_func'}


df_proj_hour = df_all2[df_all2['project_code']==test_project_code].groupby(lst_groupby).agg(dic_agg).reset_index()
df_proj_hour.rename(columns=dic_rename, inplace=True)

df_proj_hour['add_hour'] = df_proj_hour['total_hour_pro_func'] * 1
print(df_proj_hour.shape)
df_proj_hour.head(3)

(40, 5)


,project_code,sub_job_family,date,total_hour_pro_func,add_hour
0,1PD05R550001,ME,2020-01-01,331.9,331.9
1,1PD05R550001,ME,2020-02-01,2024.3,2024.3
2,1PD05R550001,ME,2020-03-01,3280.7,3280.7


In [ ]:
      df_dept_rate <- df %>%
        group_by(div, deptid, sub_job_family, date) %>%
        summarise(uti_rate = mean(utilization_rate_by_dep_func),
                  total_hour_dept_func = sum(total_hour),
                  emp_cnt = n_distinct(emplid) - sum(termination_n),
                  attendance_emp = mean(attendance)) %>%
        arrange(date) %>%
        ungroup()

In [82]:
lst_groupby = ['div', 'deptid', 'sub_job_family', 'date']
dic_agg = {'utilization_rate_by_dep_func':'mean', 'attendance':'mean',
           'total_hour':'sum', 'emplid':'nunique', 'termination_n':'sum'}

dic_rename = {'utilization_rate_by_dep_func':'uti_rate', 'attendance':'attendance_emp',
     'total_hour':'total_hour_dept_func',
     'emplid':'emp_cnt', 'attendance':'attendance_emp'}

df_dept_rate = df.groupby(lst_groupby).agg(dic_agg).rename(columns=dic_rename).reset_index()
print(df_dept_rate.shape)
df_dept_rate

(90, 9)


,div,deptid,sub_job_family,date,uti_rate,attendance_emp,total_hour_dept_func,emp_cnt,termination_n
0,23D000,23D000,RH,2020-05-01,0.000000,160,0.0,2,0
1,23D000,23D000,RH,2020-06-01,0.000000,184,0.0,2,0
2,23D000,23D000,RH,2020-07-01,0.000000,160,0.0,2,0
3,23D000,23D000,RH,2020-08-01,0.000000,200,0.0,2,0
4,23D000,23D000,RH,2020-09-01,0.000000,144,0.0,2,0
...,...,...,...,...,...,...,...,...,...
85,23D000,23D600,RH,2021-01-01,0.053125,160,8.5,1,0
86,23D000,23D600,RH,2021-02-01,0.208333,120,25.0,1,0
87,23D000,23D600,RH,2021-03-01,0.320652,184,59.0,1,0
88,23D000,23D600,RH,2021-04-01,0.545139,144,78.5,1,0


In [83]:
from datetime import datetime

date_7 = datetime.strptime("2021-07-01", "%Y-%m-%d")
date_7

datetime.datetime(2021, 7, 1, 0, 0)

In [85]:
df_dept_rate[df_dept_rate['date'] >= date_7]

,div,deptid,sub_job_family,date,uti_rate,attendance_emp,total_hour_dept_func,emp_cnt,termination_n
14,23D000,23D000,RH,2021-07-01,0.000000,160,0.000000,1,0
15,23D000,23D000,RH,2021-08-01,0.000000,200,0.000000,1,0
16,23D000,23D000,RH,2021-09-01,0.000000,144,0.000000,1,0
31,23D000,23D100,RH,2021-07-01,0.957161,160,2144.040854,14,0
32,23D000,23D100,RH,2021-08-01,0.712175,200,1994.088895,14,0
33,23D000,23D100,RH,2021-09-01,1.039462,144,2095.556301,14,0
48,23D000,23D200,RH,2021-07-01,0.719981,160,1036.772216,9,0
49,23D000,23D200,RH,2021-08-01,0.506851,200,912.332588,9,0
50,23D000,23D200,RH,2021-09-01,0.796622,144,1032.422524,9,0
65,23D000,23D300,RH,2021-07-01,0.748113,160,1675.773425,14,0
